# Eigenvalue sensitivity in a nonlinear problem

## Introduction

The vibration characteristics of linear systems do not change with the applied loads (by definition). However, real systems often do have nonlinear effects that influence their spectrum. Quantifying these sensitivities can be useful for determining design margins and for designing structures and meta-materials to have useful filtering properties. This notebook explores this idea in a simple problem and illustrates how the tools of OptimiSM and Jax can be brought to bear.

This notebook generalizes the previous example to the spectral analysis of a finitely deformed, hyperelastic structure. In doing so, we will get a glimpse at one of the nonlinear optimization-based solvers in OptimiSM. It concludes with a more interesting nonlinear eigenvalue sensitivity that requires an adjoint solve.

In [1]:
import functools
import jax
import jax.numpy as np
import numpy as onp
from scipy.sparse import linalg
from sksparse import cholmod

from optimism import Mesh
from optimism.Mesh import DofManager, EssentialBC
from optimism import EquationSolver
from optimism import FunctionSpace
from optimism import Mechanics
from optimism.material import Neohookean
from optimism import Objective
from optimism import VTKWriter
from optimism import QuadratureRule
from optimism import SparseMatrixAssembler
from optimism import Surface
from optimism import TractionBC

The geometry is the same as the in the linear example: a cantilevered plate of span $L=10$ and thickness $h = 1$. The setup procedure is nearly identical to the linear problem, so we'll pass over it without much comment. The significant changes are the following:

1. A sideset is created on the top edge of the plate, which will be used to define a traction.
2. We choose linear elements this time for comparison (actually, we omit the `elementOrder` argument which yields linear elements by default).
3. Let's keep the number of degrees of freedom the same as the linear problem, so we'll double the number of element intervals in both directions.

In [2]:
h = 1.0
L = 10.0
mesh = Mesh.construct_structured_mesh(Nx=8, Ny=80, xExtent=[0.0, h], yExtent=[0.0, L])
nodeTol = 1e-8

nodeSets = {'bottom': np.flatnonzero(mesh.coords[:,1] < 0 + nodeTol),
            'top': np.flatnonzero(mesh.coords[:,1] > L - nodeTol)}

def is_edge_on_top(xyOnEdge):
    return np.all(xyOnEdge[:,1] > L - nodeTol)

sideSets = {'top': Surface.create_edges(mesh.coords, mesh.conns, is_edge_on_top)}

mesh = Mesh.construct_mesh_from_basic_data(mesh.coords, mesh.conns, mesh.blocks, nodeSets, sideSets)

EBCs = [EssentialBC(nodeSet='bottom', field=0),
        EssentialBC(nodeSet='bottom', field=1)]

fieldShape = mesh.coords.shape
dofManager = DofManager(mesh, fieldShape, EBCs)

quadRule = QuadratureRule.create_quadrature_rule_on_triangle(degree=2)
fs = FunctionSpace.construct_function_space(mesh, quadRule)

In contrast to the linear example, we'll make the material Neo-Hookean this time to incorporate material and geometric nonlinearity:

In [3]:
E = 5.0
nu = 0.25
rho = 1.0
props = {'elastic modulus': E,
         'poisson ratio': nu,
         'density': rho}
material = Neohookean.create_material_model_functions(props)

As before, we instantiate the physics modules:

In [4]:
solidMechanics = Mechanics.create_mechanics_functions(fs, 'plane strain', material)
solidDynamics = Mechanics.create_dynamics_functions(fs, 'plane strain', material, Mechanics.NewmarkParameters)

As in the previous example, we will perform spectral analyses on the plate. However, unlike the previous example, we are going to solve a forward problem as well. In OptimiSM, physics problems are specified by writing their potential functions, which are then minimized to find the solution. We'll set up the potential now.

> 📝 Note: You might be worried that this optimization-based formulation excludes path-dependent materials and dynamics, but in fact they can be treated with time-discrete potentials. This is a subject for another day.

We will load the plate with a uniform vertical traction on the top edge so that the plate stretches in the $y$-direction. We can parameterize it by the total applied force (per unit depth) $F$. Mathematically, the traction on the top surface is thus
\begin{equation}
    \bar{\mathbf{t}}(F) = \frac{F}{h} \mathbf{e}_2,
\end{equation}
where $\mathbf{e}_2$ is the unit vector for the $y$-direction.
On the left and right edges, the traction is the zero function.

The potential energy will need to include the potential of this load, so let's create a surface quadrature rule that can integrate it:

In [5]:
surfaceQuadRule = QuadratureRule.create_quadrature_rule_1D(degree=1)

Now we can express the potential energy function. The `solidMechanics` functional does the actual integrating.

In [6]:
def compute_potential_energy(Uu, p):
    U = dofManager.create_field(Uu)
    F = p[0]
    internalVariables = p[1]
    traction_function = lambda X: np.array([0.0, F/h])
    V = TractionBC.compute_traction_potential_energy(mesh, U, surfaceQuadRule, 
                                                     mesh.sideSets['top'], traction_function)
    W = solidMechanics.compute_strain_energy(U, internalVariables)
    return W + V

We don't need the kinetic energy in our forward problem, since it's quasi-static, but it will be needed for the eigenvalue analyses, so let's code it now. Note that the `solidDynamics` module is doing all the work; all we're doing here is managing the difference between vectors of unknowns (which exclude essential boundary conditions) and the full set of degrees of freedom on the mesh.

In [7]:

def compute_kinetic_energy(Vu):
    V = dofManager.create_field(Vu)
    return solidDynamics.compute_output_kinetic_energy(V)

One last preliminary step: we will create an object of class `Objective`, which stores the objective function for our forward analysis (the potential energy) and has all the operators the solver will need on it: the gradient (which is the residual operator), the action of the hessian on vectors, and other similar things. We'll use a sparse Cholesky factorization of the stiffness matrix as the preconditioner for the linear solver (which is conjugate gradient in this case).

In [8]:
def assemble_sparse_stiffness_matrix(Uu, p):
    U = dofManager.create_field(Uu)
    internalVariables = p.state_data
    elementStiffnesses = solidMechanics.compute_element_stiffnesses(U, internalVariables)
    return SparseMatrixAssembler.assemble_sparse_stiffness_matrix(
        elementStiffnesses, mesh.conns, dofManager)

precondStrategy = Objective.PrecondStrategy(assemble_sparse_stiffness_matrix)

Uu = dofManager.get_unknown_values(np.zeros(fieldShape)) # set initial unknown displacements to zero
state = solidMechanics.compute_initial_state() # initialize internal state variables
force = 0.0 # initialize applied force
p = Objective.Params(force, state)

objective = Objective.Objective(compute_potential_energy, Uu, p, precondStrategy)


## Spectral analysis

This time, instead of explicitly computing the stiffness and mass matrices for the spectral analysis, we'll compute the action of these operations implicitly using the finite element assembly procedure.

First up is the stiffness matrix operator. Here we see a use of the `Objective` object: it already has the Hessian-on-vector operation we want, and it helpfully has instructed Jax to apply just-in-time compilation on it. The SciPy eigensolvers take in the action of matrices with its `LinearOperator` type. (The `asarray` function is needed to cast the Jax arrays into "regular" NumPy arrays. The Jax arrays carry metadata that make them incompatible with most NumPy and SciPy functions. The `asarray` operator casts to NumPy `ndarray` while avoiding a deep copy.)

In [15]:

n = dofManager.get_unknown_size()

K = linalg.LinearOperator((n, n), lambda V: onp.asarray(objective.hessian_vec(Uu, V)))


Recall that the eigenvalue solver needs the action of the inverse of $\mathbf{K}$. We'll use a conjugate gradient solver for this. One unfortunate side effect of foregoing the explicit matrix representation is that it makes it trickier to precondition the linear solves. Matrix-free preconditioning is beyond the scope of this little tutorial, so we're going to use a sparse Cholesky factorization of the stiffness matrix about the undeformed state.

For the current spectral analysis, this is more than a preconditioner - it's the exact inverse, so the CG solver isn't going to do any work. Later on when we do the spectral analysis about the deformed state, we'll reuse this factorization, which will then no longer be exact.

In [10]:

KFactor = cholmod.cholesky(assemble_sparse_stiffness_matrix(Uu, p))
approxKinv = linalg.LinearOperator((n, n), KFactor)
Kinv = linalg.LinearOperator((n, n), lambda V: linalg.cg(K, V, atol=0, tol=1e-3, M=approxKinv)[0])



Now for the mass matrix. The action $\mathbf{v} \mapsto \mathbf{M} \mathbf{v}$ can be computed by taking the derivative of the kinetic energy function. We'll use reverse mode (`jacrev`), and we'll mark it for just-in-time compilation to speed it up.

In [11]:
mass_operator = jax.jit(jax.jacrev(compute_kinetic_energy))
M = linalg.LinearOperator((n, n), lambda V: onp.asarray(mass_operator(V)))

In [12]:
    
nModes = 10
mu, modes = linalg.eigsh(A=M, k=nModes, M=K, Minv=Kinv, which='LA')
lamda = 1/mu

lamda0 = lamda[-1]
mode0 = dofManager.create_field(modes[:, -1])

TypeError: reduce() arg 2 must support iteration

Since we're at the reference configuration, the exact solution is the same as the linear elastic one. We expect nearly the same answer out of the finite element analysis, with the difference owing to the different interpolation. Furthermore, since the elements are now lower order than before, we expect the structure to be stiffer and hence the fundamental frequency to be greater.

In [ ]:
print("natural circular frequency = ", np.sqrt(lamda0))

Indeed, the frequency is a bit larger. We'll skip writing the VTK file output since it's essentially the same as before.

## Load the plate and solve for deformation

The next step is to set up the solver for the forward problem. The solvers in OptimiSM are based on optimization methods, seeking the solution field by minimizing a suitable scalar functional (here the potential energy). We'll use the default solver, which is a Steihaug trust region-conjugate gradient method, which is a robust and efficient second-order method.

Let's grab the default solver settings, which will be good for our purposes:

In [ ]:
solverSettings = EquationSolver.get_settings()


As an aside, the default settings will allow the preconditioner to be reused over multiple nonlinear iterations and trigger its update lazily - only when the linear CG solver crosses a threshold number of iterations. We can speed up the solver considerably by amortizing the cost of the preconditioner update over several nonlinear iterations. This comes at the cost of additional linear CG iterations as the system moves away from the state in which the preconditioner was updated. Since assembling and factorizing the stiffness matrix is expensive, while CG iterations are cheap, there is a range where this strategy is a net win.

Now we set $F=2$, which is fairly large (the nominal stress is $F/h = 0.4 E$), so we expect large deformations. Then we launch the solver and get back the updated nodal displacements, `Uu`. The second line is where we update the force $F$ in the parameter set.

In [ ]:

force = 2.0
p = Objective.param_index_update(p, 0, force) # put current total force in parameter set
Uu = EquationSolver.nonlinear_equation_solve(objective, Uu, p, solverSettings, useWarmStart=False)


Let's write out this state to a VTK file and visualize it.

In [ ]:
U = dofManager.create_field(Uu)
writer = VTKWriter.VTKWriter(mesh, baseFileName='nonlinear-beam-modes-deformed')
writer.add_nodal_field('displacement', U, VTKWriter.VTKFieldType.VECTORS)
writer.write()

![deformed configuration](images/nonlinear_deformed_config.png)

The black outline shows the initial, undeformed state of the plate. The deformations are rather large, as expected.

## Spectral analysis about deformed state

We repeat the spectral analysis to see how the spectrum has shifted due to the applied load. The mass matrix is invariant, but the stiffness matrix will change siginificantly due to material and geometric nonlinearity. The process here is the same as before, so we repeat the code (in an actual simulation, this would be an obvious candidate for refactoring into a function).

In [ ]:
K = linalg.LinearOperator((n, n), lambda V: onp.asarray(objective.hessian_vec(Uu, V)))
Kinv = linalg.LinearOperator((n, n), lambda V: linalg.cg(K, V, atol=0, tol=1e-9, M=approxKinv)[0])
    
nModes = 10
mu, modes = linalg.eigsh(A=M, k=nModes, M=K, Minv=Kinv, which='LA')
lamda = 1/mu
lamda0 = lamda[-1]
mode0 = modes[:, -1]

print('new circular frequency ', np.sqrt(lamda0))

writer.add_nodal_field('mode', dofManager.create_field(mode0), VTKWriter.VTKFieldType.VECTORS)
writer.write()

The fundamental frequency has increased sevenfold, which is quite dramatic. There are at least three factors shifting the frequency: (1) the Neo-Hookean model gets stiffer with increased stretch, which tends to increase the natural frequency; (2) the mode shape changes owing to geometric nonlinearity; (3) the mass moment of inertia of the plate increases, since the mass distribution is now spread further from the nodal point at the root. This last effect should tend to decrease the natural frequency. Overall, the factors on the frequency-increasing side win out.

Let's write out the modal shape to the same VTK file. We can then compose the deformation and the mode shape to see what the vibration mode looks like in the deformed configuration.

In [ ]:
writer.add_nodal_field('mode', dofManager.create_field(mode0), VTKWriter.VTKFieldType.VECTORS)
writer.write()

![deformed mode shape](images/nonlinear_vibration.gif)

The mode shape almost now looks almost like a half-symmetry model of a simply supported plate. The large load at the free end is inhibiting rotation there. 

## Sensitivity of the natural frequency to the tensile force

A designer might wish to know the marginal effect that an applied load has on the fundamental natural frequency. For example, one might want to know how fastener preload might increase resonance, or perhaps one wants to use computational design optimization on bearing pads in order to damp certain frequencies.

As before, we define the output functional
\begin{equation*}
    Q(\mathbf{u}) = \frac{ \mathbf{v}_0^T \left( \mathbf{K}(\mathbf{u}) - \lambda_0 \mathbf{M} \right) \mathbf{v}_0 }
    { \mathbf{v}_0^T \mathbf{M} \mathbf{v}_0 }.
\end{equation*}


In [ ]:
vMv = 2*compute_kinetic_energy(mode0)

def Q(Wu):
    vKv = np.dot(mode0, objective.hessian_vec(Wu, mode0))
    return (vKv - 2*lamda0*compute_kinetic_energy(mode0))/vMv


From Seth's notes, we know that the sought sensitivity of the eigenvalue is given by
\begin{equation*}
    \frac{d \lambda_0(\mathbf{u})}{d F} = \frac{d Q(\mathbf{u})}{dF}
\end{equation*}
This is not the whole story, however. What we mean by "sensitivity to $F$" is the marginal change in $\lambda_0$ with respect to applied load $F$ such that *the body remains in equilibrium*. As such, this is a PDE-constrained sensitivity problem.
We can express the sensitivity using the residual force operator for the mechanics problem $\mathbf{r}(\mathbf{u}, F)$ as
\begin{align*}
    \frac{d Q(\mathbf{u}(F))}{dF} & & \text{s.t.} & & \mathbf{r}(\mathbf{u}, F) = \mathbf{0} \; \forall F.
\end{align*}

This can be solved with the adjoint method. First, we solve the adjoint problem
\begin{equation*}
    \mathbf{K}^T \mathbf{a} = - Q_{, \mathbf{u}},
\end{equation*}
where $\mathbf{a}$ is the adjoint displacement. The comma subscript notation indicates partial differentiation with respect to the trailing subscript variable. The adjoint load follows from a simple use of reverse-mode AD. The stiffness matrix is symmetric, so we can drop the tranpose operator.

In [ ]:
adjointLoad = -jax.grad(Q)(Uu)
adjointDisplacement = linalg.cg(K, onp.array(adjointLoad), tol=1e-10, atol=0.0, M=approxKinv)[0]
adjointDisplacement = np.asarray(adjointDisplacement) # cast back to jax-numpy array


The sought sensitivities are then found from
\begin{equation*}
    \frac{d Q}{d F} = \mathbf{a}^T \mathbf{r}_{, F}.
\end{equation*}


The `Objective` class provides methods for common sensititivities, which saves us the work of writing the AD expression. Let's use it to compute the sensitivity.

In [ ]:
lamdaPrime = objective.vec_jacobian_p0(Uu, adjointDisplacement)[0]
print(lamdaPrime)

It's that simple. Notice that with the `Objective` object, which we use for our forward solve, there's almost no additional coding overhead to get adjoint sensitivities, even in a nonlinear problem.

## Verify the sensitivity

Let's do a finite difference approximation to verify this calculation.

First, increment the force and then do another solve of the forward problem:

In [ ]:
df = 1e-4
forcePlus = force + df
p = Objective.param_index_update(p, 0, forcePlus)
UuPlus = EquationSolver.nonlinear_equation_solve(objective, Uu, p, solverSettings, useWarmStart=False)

Now we'll solve the eigenvalue problem about the perturbed state.

In [ ]:
K = linalg.LinearOperator((n, n), lambda V: onp.asarray(objective.hessian_vec(UuPlus, V)))
Kinv = linalg.LinearOperator((n, n), lambda V: linalg.cg(K, V, atol=0, tol=1e-9, M=approxKinv)[0])
    
nModes = 10
mu, modes = linalg.eigsh(A=M, k=nModes, M=K, Minv=Kinv, which='LA')
lamda0Plus = 1/mu[-1]

The forward difference approximation of the frequency sensitivity is then

In [ ]:
lamdaPrimeFD = (lamda0Plus - lamda0)/df
print("finite difference", lamdaPrimeFD)

The difference is less than two parts per ten thousand.

In [ ]:
(lamdaPrime-lamdaPrimeFD)/lamdaPrimeFD